In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# Пути к файлам
labels_csv='../input/dog-breed-identification/labels.csv'
sample_submission_csv='../input/dog-breed-identification/sample_submission.csv'

# Пути до картинок
jpg_train='../input/dog-breed-identification/train/{}.jpg'  
jpg_test='../input/dog-breed-identification/test/{}.jpg'

# Настройки нейросети
im_resize = 64 # Размер изображения
num_class = 120 # Кол-во классов
batch_size = 32
Epochs = 50


In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow.keras as keras
from keras import regularizers
from keras.models import Model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.metrics import categorical_accuracy, categorical_crossentropy
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
def gen_graph(history, title):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('crossentropy ' + title)
    plt.ylabel('crossentropy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('categorical_accuracy ' + title)
    plt.ylabel('categorical_accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()


In [ ]:
df_train = pd.read_csv(labels_csv) 
df_test = pd.read_csv(sample_submission_csv)


In [ ]:
df_train.head()


In [ ]:
df_test.head()


In [ ]:
labels = df_train['breed']
one_hot = pd.get_dummies(labels, sparse = True)
one_hot_labels = np.asarray(one_hot)
#one_hot_labels


In [ ]:
x_train = []
y_train = []
x_test = []


In [ ]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = load_img(jpg_train.format(f), target_size=(im_resize, im_resize))
    img_resized = img_to_array(img)
    x_train.append(img_resized)
    label = one_hot_labels[i]
    y_train.append(label)
    i += 1


In [ ]:
for f in tqdm(df_test['id'].values):
    img = load_img(jpg_test.format(f), target_size=(im_resize, im_resize))
    img_resized = img_to_array(img)
    x_test.append(img_resized)


In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, shuffle=True,  test_size=0.2)


In [ ]:
del x_train, y_train, df_train


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255) #нормализация

test_datagen = ImageDataGenerator(rescale=1./255)

# Создаем генераторы 
train_generator =train_datagen.flow(np.array(X_train), np.array(Y_train), 
                               batch_size=batch_size)
test_generator =test_datagen.flow(np.array(X_valid), np.array(Y_valid),
                              batch_size=batch_size*5)


In [ ]:
# Создаем последовательную модель
model = Sequential()

# сверточный слой
model.add(Conv2D(32, (3, 3), padding='same',input_shape=(im_resize, im_resize, 3), activation='relu'))
# сверточный слой
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# сверточный слой
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
# сверточный слой
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
# Третий слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))


# Выходной полносвязный слой
model.add(Dense(num_class, activation='softmax'))


In [ ]:
model.compile(optimizer='adam',
          loss='categorical_crossentropy', 
           metrics=[categorical_accuracy])


In [ ]:
print(model.summary())


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

earlystop=EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

checkpoint_callback = ModelCheckpoint('model_best.hdf5',
                                      monitor='val_categorical_accuracy', 
                                      save_best_only=True,
                                      verbose=1)


In [ ]:
history = model.fit(
    train_generator,
    callbacks=[earlystop, checkpoint_callback],
    epochs=Epochs,
    steps_per_epoch=len(train_generator),
    
    validation_data=test_generator,
    validation_steps=len(test_generator))


In [ ]:
#График точности на валидационной и обучающей выборке
gen_graph(history,
          "график точности")


In [ ]:
from tensorflow.keras.models import load_model
model=load_model("model_best.hdf5")


In [ ]:
preds = model.predict(np.array(x_test))


In [ ]:
sub = pd.DataFrame(preds)
col_names = one_hot.columns.values
sub.columns = col_names
sub.insert(0, 'id', df_test['id'])
sub.head(5)


In [ ]:
sub.to_csv("output_rmsprop_aug.csv", index=False)
